In [6]:
# ================================================================
# DeepConvContext (Bi-Attention, paper-aligned) × Scheme 1:
# NVML GPU inference energy (mJ/inf) on Google Colab — single cell, no prereqs
# ================================================================
# 0) System check & deps
!nvidia-smi
!pip -q install pynvml

import os, json, time, math, pathlib, warnings, multiprocessing as mp
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# ---------------- NVML sampling + trapezoidal energy integration ----------------
import pynvml

def _nvml_sampler(stop_event, q, dev_index=0, interval=0.02):
    """Background process: sample GPU power (mW) every `interval` seconds, push (t, mW)."""
    import time, pynvml
    pynvml.nvmlInit()
    h = pynvml.nvmlDeviceGetHandleByIndex(dev_index)
    try:
        while not stop_event.is_set():
            q.put((time.perf_counter(), pynvml.nvmlDeviceGetPowerUsage(h)))
            time.sleep(interval)
    finally:
        pynvml.nvmlShutdown()

def _integrate_mJ_between(samples, t0, t1):
    """Trapezoidal integrate power (mW) over [t0, t1] → mJ."""
    if not samples: return 0.0
    samples = sorted(samples, key=lambda x: x[0])
    ts = np.array([t for t,_ in samples], dtype=np.float64)
    ps = np.array([p for _,p in samples], dtype=np.float64)
    mask = (ts >= t0) & (ts <= t1)
    ts_w, ps_w = ts[mask], ps[mask]
    if ts_w.size == 0 or ts_w[0] > t0:
        p0 = np.interp(t0, ts, ps); ts_w = np.insert(ts_w, 0, t0); ps_w = np.insert(ps_w, 0, p0)
    if ts_w[-1] < t1:
        p1 = np.interp(t1, ts, ps); ts_w = np.append(ts_w, t1); ps_w = np.append(ps_w, p1)
    return float(np.trapz(ps_w, ts_w))  # mW*s = mJ

def sample_idle_power_mW(duration_s=20.0, dev_index=0, interval=0.02, save_csv=None):
    """Measure average idle power (mW) for `duration_s` seconds."""
    q = mp.Queue(); stop = mp.Event()
    p = mp.Process(target=_nvml_sampler, args=(stop, q, dev_index, interval)); p.start()
    time.sleep(duration_s)
    stop.set(); p.join()
    samples = []
    while not q.empty(): samples.append(q.get())
    if not samples: raise RuntimeError("No NVML samples during idle.")
    samples.sort(key=lambda x: x[0])
    t0, t1 = samples[0][0], samples[-1][0]
    E_idle_mJ = _integrate_mJ_between(samples, t0, t1)
    T_idle_s  = max(1e-9, t1 - t0)
    P_idle_mW = E_idle_mJ / T_idle_s
    if save_csv:
        pd.DataFrame(samples, columns=["t_abs_s","power_mW"]).to_csv(save_csv, index=False)
    return P_idle_mW, samples

def measure_mJ_per_inference(run_once, n_items_per_call, repeats, P_idle_mW,
                             dev_index=0, interval=0.02, save_csv=None):
    """Concurrent NVML sampling + integration + idle subtraction → mJ/inf."""
    q = mp.Queue(); stop = mp.Event()
    p = mp.Process(target=_nvml_sampler, args=(stop, q, dev_index, interval)); p.start()
    t0 = time.perf_counter()
    for _ in range(repeats):
        run_once()
    t1 = time.perf_counter()
    stop.set(); p.join()
    samples = []
    while not q.empty(): samples.append(q.get())
    if not samples: raise RuntimeError("No NVML samples during active measurement.")
    E_total_mJ = _integrate_mJ_between(samples, t0, t1)
    T_total_s  = max(1e-9, t1 - t0)
    E_idle_mJ  = P_idle_mW * T_total_s
    n_inf      = max(1, repeats * n_items_per_call)
    if save_csv:
        pd.DataFrame(samples, columns=["t_abs_s","power_mW"]).to_csv(save_csv, index=False)
    return {
        "mJ_per_inf": max(0.0, (E_total_mJ - E_idle_mJ) / n_inf),
        "ms_per_inf": (T_total_s / n_inf) * 1e3,
        "throughput_inf_per_s": n_inf / T_total_s,
        "n_inferences": n_inf,
        "repeats": repeats,
        "T_total_s": T_total_s,
        "E_total_mJ": E_total_mJ,
        "E_idle_mJ": E_idle_mJ,
        "P_idle_mW": P_idle_mW,
        "t0_abs": t0, "t1_abs": t1
    }

def calibrate_repeats(run_once, target_s=8.0, min_rep=3, max_rep=5000):
    """Estimate repeats so one measurement window lasts ~target_s seconds."""
    run_once()
    t0 = time.perf_counter(); run_once(); t1 = time.perf_counter()
    dt = max(1e-4, t1 - t0)
    reps = int(np.ceil(target_s / dt))
    return int(np.clip(reps, min_rep, max_rep))

def measure_with_bootstrap(name, run_once, n_items, repeats, n_runs=5, n_boot=1000, logdir=Path("logs")):
    """Repeat n_runs, bootstrap mean mJ/inf with 95% CI; save traces and summary."""
    logdir.mkdir(exist_ok=True, parents=True)
    runs = []
    for i in range(n_runs):
        print(f"[Measure] {name} run {i+1}/{n_runs} ...")
        r = measure_mJ_per_inference(
            run_once, n_items, repeats, P_idle_mW,
            dev_index=0, interval=0.02,
            save_csv=str(logdir/f"power_trace_{name}_run{i+1}.csv")
        )
        runs.append(r)
    mJs = np.array([r["mJ_per_inf"] for r in runs], dtype=np.float64)
    rng = np.random.default_rng(123)
    boots = [float(np.mean(mJs[rng.integers(0, len(mJs), size=len(mJs))])) for _ in range(n_boot)]
    ci_low, ci_high = np.percentile(boots, [2.5, 97.5])
    summary = {"model": name, "mean_mJ_per_inf": float(mJs.mean()),
               "ci95_low": float(ci_low), "ci95_high": float(ci_high), "runs": runs}
    with open(logdir/f"energy_{name}.json", "w") as f: json.dump(summary, f, indent=2)
    print(f"[Result] {name}: {summary['mean_mJ_per_inf']:.3f} mJ/inf (95% CI [{summary['ci95_low']:.3f}, {summary['ci95_high']:.3f}])")
    return summary

# ---------------- DeepConvContext (Bi-Attention, paper-aligned) — exactly as provided ----------------
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'  # GPU memory-friendly setting

import json as _json_shadow  # avoid name shadow
import numpy as _np_shadow
from pathlib import Path as _Path_shadow
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- Basic configuration (unaltered) ---
torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("\n\nStep 10: DeepConvContext (Bi-Attention, paper-aligned RWHAR variant)")
print("=" * 90)
print(f"Device in use: {device}")

SEED = 1
def set_seed(seed: int = SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

CFG_DIR = Path('/content/configs')
CFG_DIR.mkdir(parents=True, exist_ok=True)
classes_json = CFG_DIR / 'classes.json'
splits_json  = CFG_DIR / 'splits.json'

# Auto-create minimal configs if missing (does not change model structure/params)
if not classes_json.exists():
    classes_json.write_text(json.dumps({
        "num_classes": 8,
        "standard_classes": [f"class{i}" for i in range(8)]
    }, indent=2))
if not splits_json.exists():
    splits_json.write_text(json.dumps({
        "folds": [{"fold": 0, "test_subject": "S01"}]
    }, indent=2))

with open(CFG_DIR / 'classes.json', 'r') as f:
    classes_cfg = json.load(f)
with open(CFG_DIR / 'splits.json', 'r') as f:
    splits_cfg = json.load(f)

NUM_CLASSES    = int(classes_cfg['num_classes'])
STANDARD_NAMES = classes_cfg.get('standard_classes', [str(i) for i in range(NUM_CLASSES)])

NUM_CHANNELS     = 6
SAMPLES_PER_WIN  = 150
WIN_OVERLAP      = 0.5
STRIDE_SAMPLES   = int(SAMPLES_PER_WIN * (1 - WIN_OVERLAP))  # 75

EPOCHS        = 30
LEARNING_RATE = 1e-4
WEIGHT_DECAY  = 1e-6
STEP_SIZE     = 10
GAMMA         = 0.9

DROPOUT_P        = 0.5
HIDDEN_UNITS     = 128
CONV_CHANNELS    = 64
KERNEL_SIZE      = 9
PROJECTION_DIM   = 128
ATTN_HEADS       = 4
ATTN_LAYERS      = 3
MAX_CONTEXT_WINS = 200

TRAIN_BATCH  = 100
EVAL_BATCH   = 100

# --- Model definition (unaltered) ---
class DeepConvLSTM_Intra(nn.Module):
    def __init__(self, in_ch: int = 6, conv_ch: int = 64, kernel_size: int = 9, hidden: int = 128):
        super().__init__()
        pad = kernel_size // 2
        self.conv1 = nn.Conv1d(in_ch,   conv_ch, kernel_size, padding=pad)
        self.conv2 = nn.Conv1d(conv_ch, conv_ch, kernel_size, padding=pad)
        self.conv3 = nn.Conv1d(conv_ch, conv_ch, kernel_size, padding=pad)
        self.conv4 = nn.Conv1d(conv_ch, conv_ch, kernel_size, padding=pad)
        self.relu  = nn.ReLU(inplace=True)
        self.lstm  = nn.LSTM(input_size=conv_ch, hidden_size=hidden, num_layers=1, batch_first=True)
    def forward(self, x_win: torch.Tensor) -> torch.Tensor:
        x = self.relu(self.conv1(x_win))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = x.permute(0, 2, 1)
        _, (h_n, _) = self.lstm(x)
        return h_n[-1]

class PositionalEncoding1D(nn.Module):
    def __init__(self, d_model: int, max_len: int = 200):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_len, d_model)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        seq_len = x.size(1)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0)
        pos = self.pos_embedding(positions)
        return x + pos

class MultiHeadSelfAttentionBlock(nn.Module):
    def __init__(self, dim: int, num_heads: int):
        super().__init__()
        self.mha  = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=0.0, batch_first=True)
        self.norm = nn.LayerNorm(dim)
    def forward(self, x: torch.Tensor, attn_mask: torch.Tensor | None = None) -> torch.Tensor:
        attn_out, _ = self.mha(x, x, x, attn_mask=attn_mask, need_weights=False)
        return self.norm(x + attn_out)

class DeepConvContext_BiAttention(nn.Module):
    def __init__(self,
                 num_channels: int = 6,
                 num_classes: int = 8,
                 conv_channels: int = 64,
                 hidden_intra: int = 128,
                 projection_dim: int = 128,
                 attn_heads: int = 4,
                 num_attn_layers: int = 3,
                 max_context_len: int = 200,
                 dropout: float = 0.5,
                 bidirectional: bool = True):
        super().__init__()
        self.intra = DeepConvLSTM_Intra(in_ch=num_channels, conv_ch=conv_channels, kernel_size=KERNEL_SIZE, hidden=hidden_intra)
        self.proj = nn.Linear(hidden_intra, projection_dim)
        self.pos_enc = PositionalEncoding1D(projection_dim, max_len=max_context_len)
        self.bidirectional = bidirectional
        self.attn_layers = nn.ModuleList([MultiHeadSelfAttentionBlock(projection_dim, attn_heads) for _ in range(num_attn_layers)])
        self.dropout = nn.Dropout(dropout)
        self.fc      = nn.Linear(projection_dim, num_classes)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, T = x.shape
        feats = self.intra(x)
        proj  = self.proj(feats)
        seq = proj.unsqueeze(0)
        seq = self.pos_enc(seq)
        attn_mask = None
        if not self.bidirectional:
            S_len = B
            attn_mask = torch.ones(S_len, S_len, device=x.device, dtype=torch.bool).triu(diagonal=1)
        for layer in self.attn_layers:
            seq = layer(seq, attn_mask=attn_mask)
        seq = self.dropout(seq)
        logits = self.fc(seq)
        return logits.squeeze(0)

# --- Dataset (unaltered semantics) ---
class HARWindowDataset(Dataset):
    def __init__(self, npz_file: Path, split: str = 'train'):
        data = np.load(npz_file, allow_pickle=True)
        mask = data['splits'] == split
        self.labels   = data['labels'][mask].astype(np.int64)
        self.subjects = data['subjects'][mask]
        channels = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
        self.wins = np.stack([data[ch][mask] for ch in channels], axis=1).astype(np.float32)
    def __len__(self) -> int:
        return len(self.labels)
    def __getitem__(self, idx: int):
        x = torch.from_numpy(self.wins[idx])         # (C, T)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# --- Minimal data bootstrap (only if real features are missing) ---
features_dir = Path('/content/features'); features_dir.mkdir(parents=True, exist_ok=True)
npz_path = features_dir / 'windows_normalized_fold0.npz'
if not npz_path.exists():
    N_train, N_test, T, C, K = 2000, 800, 150, 6, NUM_CLASSES
    rng = np.random.default_rng(2025)
    Xtr = rng.normal(0, 1, size=(N_train, T)).astype(np.float32)
    Xte = rng.normal(0, 1, size=(N_test,  T)).astype(np.float32)
    def make_axis(N): return rng.normal(0, 1, size=(N, T)).astype(np.float32)
    out = {
        'acc_x': make_axis(N_train+N_test), 'acc_y': make_axis(N_train+N_test), 'acc_z': make_axis(N_train+N_test),
        'gyro_x': make_axis(N_train+N_test), 'gyro_y': make_axis(N_train+N_test), 'gyro_z': make_axis(N_train+N_test),
        'labels': np.concatenate([rng.integers(0, K, size=N_train), rng.integers(0, K, size=N_test)]).astype(np.int64),
        'subjects': np.array([b'S01']*(N_train+N_test)),
        'splits': np.array(['train']*N_train + ['test']*N_test)
    }
    np.savez(npz_path, **out)

# --- Build test loader exactly like Step 10 (batch_as_context, shuffle=False) ---
test_ds  = HARWindowDataset(npz_path, split='test')
test_loader  = DataLoader(
    test_ds,
    batch_size=100,      # equals EVAL_BATCH in Step 10
    shuffle=False,
    drop_last=False,     # do not drop the last batch at test time
    num_workers=2,
    pin_memory=True,
    persistent_workers=False
)

# --- Instantiate the model with the exact hyperparameters from Step 10 ---
model = DeepConvContext_BiAttention(
    num_channels=NUM_CHANNELS,
    num_classes=NUM_CLASSES,
    conv_channels=CONV_CHANNELS,
    hidden_intra=HIDDEN_UNITS,
    projection_dim=PROJECTION_DIM,
    attn_heads=ATTN_HEADS,
    num_attn_layers=ATTN_LAYERS,
    max_context_len=MAX_CONTEXT_WINS,
    dropout=DROPOUT_P,
    bidirectional=True
).to(device).eval()

# --- Optional: load trained weights if available (does not affect energy/FLOPs) ---
models_dir = Path('/content/models'); models_dir.mkdir(exist_ok=True)
wpath = models_dir / 'deepconvcontext_biattn_rwhar_fold0.pth'
if wpath.exists():
    try:
        model.load_state_dict(torch.load(wpath, map_location=device, weights_only=True))
        print(f"[Info] Loaded weights: {wpath.name}")
    except Exception as e:
        print(f"[Warn] Failed to load weights: {e}")

# ---------------- Build run_once: one full pass over the test set ----------------
def make_runner(model: nn.Module, loader: DataLoader):
    """One complete forward over test_loader; return (callable, n_items)."""
    n_items = len(loader.dataset)
    @torch.no_grad()
    def run_once():
        for x, _ in loader:
            x = x.to(device, non_blocking=True)
            _ = model(x)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
    return run_once, n_items

run_once, N_items = make_runner(model, test_loader)

# ---------------- Measure idle power ----------------
logs_dir = Path('/content/logs'); logs_dir.mkdir(parents=True, exist_ok=True)
print("\n[Info] Sampling idle power for 20 s ...")
P_idle_mW, _idle = sample_idle_power_mW(duration_s=20.0, dev_index=0, interval=0.02,
                                        save_csv=str(logs_dir/'power_idle_trace_dcc.csv'))
print(f"[Info] Mean idle power ~ {P_idle_mW:.1f} mW")

# ---------------- Warmup & calibrate repeats (target ≥ 8 s per measurement) ----------------
print("\n[Warmup] warmup ...")
run_once(); run_once()
repeats = calibrate_repeats(run_once, target_s=8.0, min_rep=3, max_rep=5000)
print(f"[Info] repeats = {repeats} (items per call = {N_items})")

# ---------------- NVML measurement + bootstrap CI ----------------
summary = measure_with_bootstrap(
    name="deepconvcontext_biattn_rwhar_inference",
    run_once=run_once,
    n_items=N_items,
    repeats=repeats,
    n_runs=5,
    n_boot=1000,
    logdir=logs_dir
)

# ---------------- Save a compact CSV summary ----------------
df = pd.DataFrame([{
    "model": "DeepConvContext (Bi-Attn, paper-aligned)",
    "mJ_per_inf_mean": summary["mean_mJ_per_inf"],
    "ci95_low": summary["ci95_low"],
    "ci95_high": summary["ci95_high"],
    "items_per_call": N_items,
    "repeats": repeats,
    "idle_mW": P_idle_mW
}])
df.to_csv(logs_dir/"energy_summary_deepconvcontext_biattn.csv", index=False)
print("\n=== Done: GPU inference energy (Scheme 1) ===")
print(df)
print("\nArtifacts:")
print("- logs/power_idle_trace_dcc.csv")
print("- logs/power_trace_deepconvcontext_biattn_rwhar_inference_run*.csv")
print("- logs/energy_deepconvcontext_biattn_rwhar_inference.json")
print("- logs/energy_summary_deepconvcontext_biattn.csv")

Mon Nov 17 16:33:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P0             31W /   70W |     530MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----